<font color=darkgreen>
    
# Lecture 9. Introduction to Recurrent Neural Network

## 9.1 Example of sequence data

#### Sequence-to-vector 


- 예: Sentiment classification 
    <img src="figures/sentiment.png" width="50%">
   
    - Input: 단어(또는 문자)를 순차적으로 입력
    - Output: 마지막 단어가 입력된 시점에 전체 문장의 감성을 분류
     <img src="figures/seq2one.PNG" width="30%">

#### Sequence-to-sequence
- 예: Music generation 
    <img src="figures/music.PNG" width="50%">

    - Input: 20개의 순차적인 음정, 속도 등의 값 입력
    - Output: 이어지는 21번째 음정, 속도 값을 출력 
    <img src="figures/seq2seq.PNG" width="30%">


#### Delayed sequence-to-sequence
- 예: Machine translation
    <img src="figures/translation.PNG" width="50%">

    - Input: 프랑스어 문장 전체를 단어 단위로 순차적 입력
    - Output: 번역된 영어 문장을 단어 단위로 순차적 출력
    <img src="figures/seq2seq_delay.PNG" width="30%">


#### Vector-to-sequence
- 예: Image captioning 
    <img src="figures/imagecaption.png" width="50%">

    - Input: 이미지
    - Output: 이미지에 대한 설명을 단어 단위로 순차적으로 출력 
    <img src="figures/one2seq.PNG" width="30%">



## 9.2 Recurrent neural networks
### Example
- 문장을 입력하여 각 단어가 사람이름을 나타내는지 여부를 구분 
- Tokenize
    - 문장을 모형의 입력단위인 단어(혹은 음절)로 쪼개는 작업
- Input: 단어
- Output: 1 if 사람이름, 0 elsewhere


<img src="figures/entity_exam.PNG" width="50%">

- Vocabulary(dictionary) 구성 
    - 데이터 처리 시 사용할 모든 단어의 집합 (예, 10,000개의 단어 집합)
    - 각 단어를 $10000\times 1$ 크기의 one-hot vector로 표현 
<img src="figures/vocabulary.PNG" width="60%">
<img src="figures/vocab2.png" width="60%">


### Recurrent neuron
- 현 시점의 input과 앞 시점의 hidden state를  동시에 input으로 받는 neuron
- 앞에 나온 단어가 현재 단어의 사람이름 여부에 영향을 미침

<img src="figures/recurrent_neuron.PNG" width="40%">




$$ h_t = \phi (x_t^T W_x + h_{t-1}^T W_h + b)$$

### Recurrent neural networks
- Recurrent neuron을 여러 개로 하나의 layer를 구성 
<img src="figures/recurrent_layer.PNG" width="50%">

- 여러 개의 layer를 쌓아서 stacked RNN을 구성 
<img src="figures/stacked_recurrent_layer.PNG" width="50%">


### Single RNN layer with Keras
- `input_shape=(timesteps, input_features)`
    - $10000\times 1$의 one-hot vector로 표현되는 순차적인 3개 단어를 input 
    - (Harry, Potter, and) -> 0
    - (Potter, and, Hermione) -> 1
- `return_sequences=False`
    - 마지막 시점의 output 만을 출력 
    - `(batch_size, output_features)`

In [1]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(32, input_shape=(3,10000), return_sequences=False))
model.add(Dense(1))
model.summary()


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 32)                321056    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 321,089
Trainable params: 321,089
Non-trainable params: 0
_________________________________________________________________


$$ h_t = \phi (x_t^T W_x + h_{t-1}^T W_h + b)$$
- $W_x$: 10000개의 input 값과 32개 output($h_t$)과의 연결 = $32 \times 10000 = 320000$
- $W_h$: 32개 $h_{t-1}$과 32개 $h_{t}$와의 연결 = $32 \times 32 = 1024$ 
- $b$: 32개 bias

=> 320000 + 1024 + 32 = 321056

#### Stacked RNN layers with Keras

- `return_sequences=True`
    - 매 시점의 output을 출력 
    - `(batch_size, timesteps, output_features)`
    - 다음 layer의 recurrent neuron에 입력하기 위해서는 매 시점의 output이 필요

In [2]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(32, input_shape=(3,10000), return_sequences=True))
model.add(SimpleRNN(32, input_shape=(3,10000), return_sequences=False))
model.add(Dense(1))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 3, 32)             321056    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 323,169
Trainable params: 323,169
Non-trainable params: 0
_________________________________________________________________


$$ h_t = \phi (x_t^T W_x + h_{t-1}^T W_h + b)$$
- RNN layer 1
    - $W_x$: 10000개의 input 값과 32개 output($h_t$)과의 연결 = $32 \times 10000 = 320000$
    - $W_h$: 32개 $h_{t-1}$과 32개 $h_{t}$와의 연결 = $32 \times 32 = 1024$ 
    - $b$: 32개 bias
    
    => 320000 + 1024 + 32 = 321056
- RNN layer 2
    - $W_x$: 32개의 input 값과 32개 output($h_t$)과의 연결 = $32 \times 32 = 1024$
    - $W_h$: 32개 $h_{t-1}$과 32개 $h_{t}$와의 연결 = $32 \times 32 = 1024$ 
    - $b$: 32개 bias
    
    => 1024 + 1024 + 32 = 2080


## 9.3 The Problem of Long-Term Dependencies
- 이론적으로 RNN은 예전 입력값을 무한히 기억할 수 있음
- 학습과정에서 gradient가 불안정해져서 가중치의 학습이 잘 되지 못함
    - Vanishing gradient문제, exploding gradient문제
    - 처음의 input값이 점점 잊혀지는 현상 발생 
- ReLU activation, parameter initialization의 조정 등 보다 모형의 구조적으로 해결하려는 시도 
    - Long Short Term Memory(LSTM; Hochreiter & Schmidhuber, 1997)
    - Gated Recurrent Unit(GRU; Kyunghyun Cho et al., 2014) 


## 9.3.1  LSTM cell
- 과거 입력값이 무조건 동일한 변환을 통해 전달되는 것이 아니라 필요에 따라 변환 없이 그대로 전달
    - f: forget gate, 이전 memory cell값을 얼마나 통과? 
    - i: input gate, 새로운 정보를 얼마나 통과?
    - o: 현재 memory cell 값을 얼마나 외부 네트워크로 통과?
    - g: 현재 memory cell의 후보값 
    
<img src="figures/lstm.PNG" width="40%" align="left">
<img src="figures/lstm2.png" width="50%">

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(32, input_shape=(3,10000), return_sequences=False))
model.add(Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                1284224   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,284,257
Trainable params: 1,284,257
Non-trainable params: 0
_________________________________________________________________


## 9.3.2 GRU cell
- LSTM cell의 단순화되었지만 성능 좋음
- Cell state와 hidden state를 결합
- Input gate와 forget gate의 결합

<img src="figures/gru.PNG" width="40%">

In [7]:
from keras.models import Sequential
from keras.layers import GRU, Dense

model = Sequential()
model.add(GRU(32, input_shape=(3,10000), return_sequences=False))
model.add(Dense(1))

## 9.3.3 Bidirectional RNN cell
    He said, “Teddy bears are on sale!”
    He said, “Teddy Roosevelt was a great President!”
- "Teddy"가 사람이름을 나타내는지 여부는 앞보다 뒤의 단어들이 영향을 줌
- 시간 순서대로 적용한 RNN과 시간 역순대로 적용한 RNN을 합쳐서(concat, ave 등) 구성
<img src="figures/bidirectional_rnn.PNG" width="30%">

In [8]:
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense
model = Sequential()
model.add(Bidirectional(LSTM(32, return_sequences=True),
                        input_shape=(3, 10000)))
model.add(Dense(1, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 3, 64)             2568448   
_________________________________________________________________
dense_6 (Dense)              (None, 3, 1)              65        
Total params: 2,568,513
Trainable params: 2,568,513
Non-trainable params: 0
_________________________________________________________________
